# Project 80: Path sum: two ways

In a 5 by 5 matrix below, the minimal path sum from the top left to the bottom right, by only moving to the right and down, is the path with the lowest sum. 

Find the minimal path sum, in matrix.txt. As a 31K text file containing a 80 by 80 matrix, from the top left to the bottom right by only moving right and down.

# Solution

This is an almost text book problem for Djikstra's algorithm. We can intrepret the element of each matrix as a node connected by uni-directional paths going right and down. The distance of each path is simply the value of the element it lands on. 

A final consideration is that of the 00 element, i.e top-left. This is an additional value to add to the length of the path calculated to Djikstra's algorithm. 

In [136]:
from collections import defaultdict

class Graph:
    def __init__(self):
        self.nodes = set()
        self.edges = defaultdict(list)
        self.distance = {}

    def add_node(self, value):
        self.nodes.add(value)

    def add_edge(self, from_node, to_node, distance):
        assert from_node in self.nodes, "ERROR: starting value must be a node. "
        assert to_node in self.nodes, "ERROR: starting value must be a node. "

        self.edges[from_node].append(to_node)
        self.distance[(from_node, to_node)] = distance

In [192]:
import numpy as np
from collections import OrderedDict

def martix_to_graph(mat):
    """
        Turns a matrix into a graph with each element connect to the 
        one to the right and one below it. 
        
        A graph is stored as an order dictionary, such that each node is a dictionary
    """
    graph = Graph()
    # First set up all the first element 
    for x in xrange(mat.shape[0]):
        for y in xrange(mat.shape[1]):
            graph.add_node((x,y))
        
    # Now create the connections 
    for x in xrange(mat.shape[0]-1):
        for y in xrange(mat.shape[1]-1):
            # Downward edges
            graph.add_edge( (x,y),(x+1,y), mat[x+1,y])
            graph.add_edge( (x,y),(x,y+1), mat[x,y+1])
            
            graph.add_edge( (x+1,y),(x,y), mat[x+1,y])
            graph.add_edge( (x,y+1),(x,y), mat[x,y+1])
    return graph

Load the graph from the file

In [194]:
test = np.array([[1,2,3],
                [4,5,6]])
mat   = np.loadtxt("matrix.txt", delimiter=',')
graph = martix_to_graph(test)

## Dijkstra's Algorithm

Let the node at which we are starting be called the initial node. Let the distance of node Y be the distance from the initial node to Y. Dijkstra's algorithm will assign some initial distance values and will try to improve them step by step.

* Assign to every node a tentative distance value: set it to zero for our initial node and to infinity for all other nodes.


* Set the initial node as current. Mark all other nodes unvisited. Create a set of all the unvisited nodes called the unvisited set.

* For the current node, consider all of its neighbors and calculate their tentative distances. Compare the newly calculated tentative distance to the current assigned value and assign the smaller one. For example, if the current node A is marked with a distance of 6, and the edge connecting it with a neighbor B has length 2, then the distance to B (through A) will be 6 + 2 = 8. If B was previously marked with a distance greater than 8 then change it to 8. Otherwise, keep the current value.

* When we are done considering all of the neighbors of the current node, mark the current node as visited and remove it from the unvisited set. A visited node will never be checked again.
If the destination node has been marked visited (when planning a route between two specific nodes) or if the smallest tentative distance among the nodes in the unvisited set is infinity (when planning a complete traversal; occurs when there is no connection between the initial node and remaining unvisited nodes), then stop. The algorithm has finished.

* Otherwise, select the unvisited node that is marked with the smallest tentative distance, set it as the new "current node", and go back to step 3.


In [214]:
from copy import copy

def find_minimum_path(graph,start_node,final_node):
    """"""
    visited = [] # The set of all nodes that have been visited 
    unvisted_nodes = {}
    for node in graph.nodes:
        unvisted_nodes[node] = 1e10
    unvisted_nodes[start_node] = 0
    
    current_node = start_node 
    paths = defaultdict(list)
    path_histories = []
    
    while unvisted_nodes:    
        # Store what every tentative path looks like. 
        path_histories.append(paths)

        # Add the current node to the path 
        paths[current_node].append(current_node)
        
        # Evaluate how far we have travelled 
        current_distance = unvisted_nodes[current_node] 
        
        # Consider all neighbours connect - recall: 
        # edges is a dictionary of nodes and distances 
        for neighbour in graph.edges[current_node]:
            # We never revist visited nodes
            if neighbour in visited:
                continue 
                
            # Calculate the new distance this path would have  
            tentative_distance = current_distance + graph.distance[(current_node,neighbour)]
            if tentative_distance < unvisted_nodes[neighbour]:
                paths[neighbour] = copy(paths[current_node])
                paths[neighbour].append(neighbour)
                unvisted_nodes[neighbour] = tentative_distance

        # Now that we're done marking all neighbours
        # remove the current node and progress along 
        del unvisted_nodes[current_node] 
        # Mark it as visited
        visited.append(current_node)
        # Move to the node with the smallest distance
        current_node = min(unvisted_nodes, key=unvisted_nodes.get) 

        # If that node is the one
        if current_node == final_node:
            break

    # Don't forget to update the total distsance with the distance to the final node        
    current_distance += unvisted_nodes[final_node]
    return paths[final_node],current_distance,paths,visited,path_histories

In [215]:
print find_minimum_path(graph, (0,0),(0,2))

([(0, 0), (0, 1), (0, 1), (0, 2)], 9, defaultdict(<type 'list'>, {(0, 1): [(0, 0), (0, 1), (0, 1)], (1, 0): [(0, 0), (1, 0), (1, 0)], (0, 0): [(0, 0)], (1, 1): [(0, 0), (0, 1), (0, 1), (1, 1)], (0, 2): [(0, 0), (0, 1), (0, 1), (0, 2)]}), [(0, 0), (0, 1), (1, 0)], [defaultdict(<type 'list'>, {(0, 1): [(0, 0), (0, 1), (0, 1)], (1, 0): [(0, 0), (1, 0), (1, 0)], (0, 0): [(0, 0)], (1, 1): [(0, 0), (0, 1), (0, 1), (1, 1)], (0, 2): [(0, 0), (0, 1), (0, 1), (0, 2)]}), defaultdict(<type 'list'>, {(0, 1): [(0, 0), (0, 1), (0, 1)], (1, 0): [(0, 0), (1, 0), (1, 0)], (0, 0): [(0, 0)], (1, 1): [(0, 0), (0, 1), (0, 1), (1, 1)], (0, 2): [(0, 0), (0, 1), (0, 1), (0, 2)]}), defaultdict(<type 'list'>, {(0, 1): [(0, 0), (0, 1), (0, 1)], (1, 0): [(0, 0), (1, 0), (1, 0)], (0, 0): [(0, 0)], (1, 1): [(0, 0), (0, 1), (0, 1), (1, 1)], (0, 2): [(0, 0), (0, 1), (0, 1), (0, 2)]})])


In [204]:
print graph.distance[((0,0),(0,1))]

2
